In [1]:
# Set the working directory
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

import numpy as np

Using TensorFlow backend.


In [2]:
# Initialize the CNN
classifier = Sequential()

# Add the convolution layer
classifier.add(Convolution2D(32, (3, 3), strides=(1,1), input_shape=(64, 64, 3), activation='relu'))

# Add the pooling layer
classifier.add(MaxPool2D(pool_size=(2,2)))

# Add a second convolution layer
classifier.add(Convolution2D(32, (3, 3), strides=(1,1), activation='relu'))

# Add the pooling layer
classifier.add(MaxPool2D(pool_size=(2,2)))

# Add flattern layer
classifier.add(Flatten())

# Add fully connected
classifier.add(Dense(units=128, activation='relu'))

# Add output unit
classifier.add(Dense(units=1, activation='sigmoid'))

In [3]:
def build_cnn(layer, channel, filter_size, dense_u, opt='adam'):
    clf = Sequential()
    
    clf.add(Convolution2D(channel, (filter_size,filter_size), strides=(1,1), input_shape=(64,64,3), activation='relu'))
    clf.add(MaxPool2D(pool_size=(2,2)))
    
    for i in range(layer-1):
        clf.add(Convolution2D(channel, (filter_size,filter_size), strides=(1,1), activation='relu'))
        clf.add(MaxPool2D(pool_size=(2,2)))
    
    clf.add(Flatten())
    clf.add(Dense(units=dense_u, activation='relu', kernel_initializer='glorot_uniform'))
    clf.add(Dense(units=1, activation='sigmoid', kernel_initializer='glorot_uniform'))
    
    clf.compile(optimizer=opt, loss = 'binary_crossentropy', metrics=['accuracy'])
                
    return clf

In [4]:
from IPython.display import display
from PIL import Image

In [12]:
# Image preprocessing and augmentation
from keras.preprocessing.image import ImageDataGenerator

def train_clf(layer=4, channel=32, filter_size=3, dense_u=128, opt='adam', nb_epoch=10):
    
    classifier = build_cnn(layer=layer, channel=channel, filter_size=filter_size, dense_u=dense_u, opt=opt)
    
    train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True
                )

    test_datagen = ImageDataGenerator(rescale=1./255)

    training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                    target_size=(64,64),
                                                    batch_size=40,
                                                    class_mode='binary')

    test_set = test_datagen.flow_from_directory('dataset/test_set',
                                               target_size=(64,64),
                                               batch_size=32, 
                                               class_mode='binary')
    hist = classifier.fit_generator(training_set,
                             steps_per_epoch=30,
                             nb_epoch=nb_epoch,
                             validation_data=test_set,
                             validation_steps=10, verbose=0)
    
    
    return np.mean(hist.history['val_loss'][-10:])

In [13]:
from sklearn.model_selection import ParameterGrid

param = {
    'layer': [3, 4, 5, 6],
    'channel': [32, 64],
    'filter_size': [3, 5],
    'dense_u': [128, 256],
    'opt': ['adam', 'RMSprop'],
}

parameters = list(ParameterGrid(param))

In [14]:
loss_track = np.zeros(len(parameters))

for i in range(len(parameters)):
    loss_track[i] = train_clf(**parameters[i])

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:30: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:30: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=10, validation_steps=10, verbose=0, steps_per_epoch=30)`


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


ValueError: Negative dimension size caused by subtracting 3 from 2 for 'conv2d_27/convolution' (op: 'Conv2D') with input shapes: [?,2,2,32], [3,3,32,32].

In [ ]:
test.history['val_loss'][-1]

In [ ]:
# Predict a given image
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result=classifier.predict(test_image)

if result[0][0] == 1:
    print('dog')
else:
    print('cat')